In [1]:
# Install required packages
!pip install -qU transformers datasets wandb accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/arabic-history-questions-dataset/all_data_flattened.json
/kaggle/input/arabic-history-questions-dataset/all_data.json
/kaggle/input/arabic-history-questions-dataset/train.json
/kaggle/input/arabic-history-questions-dataset/test.json
/kaggle/input/arabic-history-questions-dataset/test_raw.json
/kaggle/input/arabic-history-questions-dataset/val_raw.json
/kaggle/input/arabic-history-questions-dataset/train_raw.json
/kaggle/input/arabic-history-questions-dataset/val.json


In [3]:
# Import libraries
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq
)
from datasets import DatasetDict, Dataset
# from huggingface_hub import notebook_login
import wandb

2025-05-09 22:05:46.154046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746828346.345418      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746828346.404090      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # Login to Hugging Face Hub
# notebook_login()

wandb.login(key='77eafacbf29d3f89b810de78fe1f766a9b6e6fe8')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: noureenaboarab2003 (noureenaboarab2003-ain-shams-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
# Load dataset function
def load_data(train_path, val_path):
    print("🔄 Loading datasets...")
    train_data = pd.read_json(train_path)
    val_data = pd.read_json(val_path)
    
    train_data.rename(columns={"paragraph": "context"}, inplace=True)
    val_data.rename(columns={"paragraph": "context"}, inplace=True)
    
    print("\n📊 Training Data Sample:")
    print(train_data.head(1).T)
    print("\n📊 Validation Data Sample:")
    print(val_data.head(1).T)
    
    return train_data, val_data
    
# train_path = "/kaggle/input/train.json"
# val_path = "/kaggle/input/val.json"
train_path = "/kaggle/input/arabic-history-questions-dataset/train.json"
val_path = "/kaggle/input/arabic-history-questions-dataset/val.json"

train_data, val_data = load_data(train_path, val_path)

🔄 Loading datasets...

📊 Training Data Sample:
                                                          0
context   ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من ا...
question  ما هو الغرض الرئيسي من الممر الذي يتفرع إلى به...
answer                                   للاستقبال والترفيه

📊 Validation Data Sample:
                                                          0
context   كانت الحكومة المصرية على عهد محمد على حكومة مط...
question         ما الفرق بين حكومة محمد علي وعصر المماليك؟
answer                                 وجود نظام إداري منظم


In [6]:
# Format dataset
def format_for_qg(dataset, dataset_name="train"):
    print(f"\n🔍 Formatting {dataset_name} dataset...")
    formatted_text =dataset["context"]
    
    print("\n📝 Sample Formatted Input (first 200 chars):")
    for i in range(3):
        print(f"Sample {i+1}: {formatted_text.iloc[i][:200]}...")
    
    return pd.DataFrame({
        "text": formatted_text,
        "question": dataset["question"]
    })

train_qg = format_for_qg(train_data, "train")
val_qg = format_for_qg(val_data, "validation")


🔍 Formatting train dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من الصناع، وكانت أكثر اتساعًا، ويشمل كل منزل فناء مكشوفًا وأربع حجرات أبوابها مفتحة عليها، وتتصل بخمس حجرات أخرى، وكانت الحجرات جميعها مسقوفه بقوائم )عروق(من ...
Sample 2: القصر الفرعونى: تميز القصر الفرعونى بالاتساع والفخامة، وكان يسمى بـ)البيت العظيم، ( وقُسم إلى قسمين: قسم عام يستقبل فيه الأشراف وكبار رجال الدولة، وقسم خاص يشمل القاعات الخاصة، وكانت تعتمد على أسقف أع...
Sample 3: توفي إمبراطور قوانغشو في ١٤ نوفمبر ١٩٠٨ ، وفي ١٥ نوفمبر ١٩٠٨ ، توفي تسيشي أيضا . وزعمت الشائعات أنها أو يوان شيكاي أمرت الخصيان الموثوق بهم بتسميم إمبراطور قوانغشو ، وأكد تشريح الجثة الذي أجري بعد ما ...

🔍 Formatting validation dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: كانت الحكومة المصرية على عهد محمد على حكومة مطلقة تسود فىها قاعدة حكم الفرد، ولكن الفارق بينها وبين ما كانت عليه فى عصر المماليك أن محمد على باشا وضع نظاما لإدارتها، فحل هذا النظام محل الفوضى

In [7]:
# Convert to DatasetDict
datasets_qg = DatasetDict({
    "train": Dataset.from_pandas(train_qg),
    "validation": Dataset.from_pandas(val_qg)
})

In [8]:
datasets_qg

DatasetDict({
    train: Dataset({
        features: ['text', 'question'],
        num_rows: 4702
    })
    validation: Dataset({
        features: ['text', 'question'],
        num_rows: 550
    })
})

In [9]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration


# # Load the model and tokenizer
# model_name = "google/flan-t5-base"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)


# # Add <sep> token
# special_tokens = {'additional_special_tokens': ['<sep>']}
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# # Verify tokenization
# sample_text = "السؤال: متى تأسست السعودية؟ <sep> الجواب: 1932 <sep> السياق: تأسست المملكة..."
# print("\n🔍 Tokenization Test:")
# print("Tokens:", tokenizer.tokenize(sample_text)[:20])
# print("<sep> token ID:", tokenizer.convert_tokens_to_ids("<sep>"))

In [10]:
from transformers import (
    MT5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

In [11]:
# Load AraBART-finetuned-wiki-ar
print("\n🔄 Loading MT5 model...")
model_name = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

printed_samples = 0  # Global variable

# Tokenization function optimized for AraBART
def tokenize_function(examples):
    global printed_samples  # Access the global variable
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["question"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    
# Print first 5 samples
    for i in range(len(model_inputs["input_ids"])):
        if printed_samples < 5:
            print(f"\n🔢 Sample {printed_samples + 1}:")
            print("Input:", tokenizer.decode(model_inputs["input_ids"][i][:20]))
            print("Label:", tokenizer.decode(labels["input_ids"][i][:10]))
            printed_samples += 1
        else:
            break
    
    return model_inputs

tokenized_qg_datasets = datasets_qg.map(tokenize_function,
                                  batched=True,
                                  batch_size=8,
                                  remove_columns=["text", "question"])


🔄 Loading MT5 model...


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



🔢 Sample 1:
Input: ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من الصناع
Label: ما هو الغرض الرئيسي من الممر الذي

🔢 Sample 2:
Input: القصر الفرعونى: تميز القصر الفرعونى بالاتساع والفخامة،
Label: أين كانت تقع غرفة النو

🔢 Sample 3:
Input: توفي إمبراطور قوانغشو في ١٤ نوفمبر ١٩٠٨ ، وفي
Label: ما الذي جعل الحكومة التي أن

🔢 Sample 4:
Input: ظهر الدور العلمى للمسجد مع بداية تأسيس النبي صلى الله عليه وسلم
Label: ما هي الأهداف التعليمية للكتاتيب

🔢 Sample 5:
Input: كانت الأسرة الغنية تستأجر مرضعات لأطفالهم، ويبعث
Label: ما هي المواد الدراسية التي كان 


Map:   0%|          | 0/550 [00:00<?, ? examples/s]

In [12]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, GenerationConfig

# # model_name = "UBC-NLP/AraT5-base"
# model_name = "Abdou/arabic-tashkeel-flan-t5-small"

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# data_collator = DataCollatorForSeq2Seq(
#     tokenizer=tokenizer,
#     padding=True,  # Dynamically pad to the longest in the batch
#     label_pad_token_id=-100  # Ensures padded tokens in labels don't affect training
# )

# def tokenize_function(examples):
#     # return tokenizer(
#     #     examples["text"],  # Input: Context + Answer
#     #     text_target=examples["required"],
#     #     # padding="max_length",
#     #     padding= True,
#     #     truncation=True,
#     #     label_pad_token_id=-100 
#     # )
#     model_inputs = tokenizer(
#         examples["text"],  # Input: Context
#         max_length=768,  # Limit input length
#         truncation=True,
#         padding="max_length",  # Ensures uniform input size
#     )

#     labels = tokenizer(
#         examples["required"],  # Output: Target
#         max_length=100,  # Target length is much shorter
#         truncation=True,
#         padding="max_length"  # Ensures uniform output size
#     )

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_qg_datasets = datasets_qg.map(tokenize_function, batched=True, remove_columns=["text", "required"])
# # tokenized_ag_datasets = datasets_ag.map(tokenize_function, batched=True, remove_columns=["text", "required"])


In [13]:
print(type(tokenized_qg_datasets))  # Should be <class 'datasets.dataset_dict.DatasetDict'>
print(tokenized_qg_datasets)  # Prints dataset details
print(tokenized_qg_datasets["train"][0])


<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4702
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 550
    })
})
{'input_ids': [259, 2218, 5643, 66118, 31858, 13790, 402, 25191, 842, 267, 259, 12563, 548, 89539, 445, 105046, 122506, 556, 402, 41887, 343, 341, 12563, 1021, 9674, 259, 161017, 21128, 1093, 632, 343, 259, 152997, 4326, 2042, 259, 20380, 1189, 25543, 40730, 140296, 1093, 632, 341, 96178, 16208, 7123, 32339, 184000, 259, 131419, 7139, 5587, 632, 343, 105662, 5722, 63577, 18527, 16208, 7123, 259, 28240, 842, 343, 341, 12563, 52949, 7123, 259, 9793, 913, 5643, 149673, 16858, 24320, 974, 12306, 259, 271, 1923, 152029, 312, 3691, 7360, 60400, 343, 341, 12563, 548, 33478, 259, 9793, 94265, 52864, 1491, 92209, 445, 343, 341, 89539, 636, 259, 53442, 12592, 16208, 7123, 402, 20380, 78557, 70986, 9554, 41216, 3

In [14]:
# Function to decode and print input and labels
def print_decoded_example(dataset, split, index):
    print(f"\nExample from {split} split (index {index}):")
    print("Decoded input:", tokenizer.decode(dataset[split][index]["input_ids"]))
    print("Decoded labels:", tokenizer.decode(dataset[split][index]["labels"]))
    print("-" * 50)

# Print examples from train, validation, and test splits
print_decoded_example(tokenized_qg_datasets, "train", 200)  # First example from training set
print_decoded_example(tokenized_qg_datasets, "validation", 2)  # First example from validation set



Example from train split (index 200):
Decoded input: وقد تميز فن النحت الرومانى بمميزات عديدة أهمها: ١. الواقعية الشديدة: مثل رأس قيصر المصنوع من حجر البازلت وتمثال بومبى. وتماثيل أغسطس. ٢. الابتكار والتجديد: وخاصةً فى نحت التماثيل الشخصية وكذلك الفنون المعمارية ومنها: معبد البانثييون، وأقواس النصر والأعمدة التذكارية.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [15]:
# Function to check special tokens
def check_special_tokens(tokenizer):
    print("\nSpecial tokens:")
    print(f"<context>: {tokenizer.convert_tokens_to_ids('<context>')}")
    print(f"<answer>: {tokenizer.convert_tokens_to_ids('<answer>')}")
    print(f"<question>: {tokenizer.convert_tokens_to_ids('<question>')}")
    print(f"</s>: {tokenizer.convert_tokens_to_ids('</s>')}")

In [16]:
# Define special tokens
special_tokens_dict = {"additional_special_tokens": ["<context>", "<answer>", "<question>"]}

# Add them to the tokenizer
tokenizer.add_special_tokens(special_tokens_dict)

# Check again
check_special_tokens(tokenizer)



Special tokens:
<context>: 250100
<answer>: 250101
<question>: 250102
</s>: 1


In [17]:
# Check for NaN or inf in the dataset
import numpy as np

def check_for_invalid_values(dataset):
    for split in dataset:
        print(f"Checking {split} split...")
        for example in dataset[split]:
            if np.isnan(example["input_ids"]).any() or np.isinf(example["input_ids"]).any():
                print(f"Invalid input_ids found in {split} split!")
            if np.isnan(example["labels"]).any() or np.isinf(example["labels"]).any():
                print(f"Invalid labels found in {split} split!")

check_for_invalid_values(tokenized_qg_datasets)

Checking train split...
Checking validation split...


In [18]:
# Check special tokens
print("Special tokens:")
print(f"<context>: {tokenizer.convert_tokens_to_ids('<context>')}")
print(f"<answer>: {tokenizer.convert_tokens_to_ids('<answer>')}")
print(f"<question>: {tokenizer.convert_tokens_to_ids('<question>')}")
print(f"</s>: {tokenizer.convert_tokens_to_ids('</s>')}")

# Decode a few examples to verify tokenization
for i in range(3):
    print(f"\nExample {i + 1}:")
    print("Input IDs:", tokenized_qg_datasets["train"][i]["input_ids"])
    print("Decoded Input:", tokenizer.decode(tokenized_qg_datasets["train"][i]["input_ids"]))
    print("Labels:", tokenized_qg_datasets["train"][i]["labels"])
    print("Decoded Labels:", tokenizer.decode(tokenized_qg_datasets["train"][i]["labels"]))

Special tokens:
<context>: 250100
<answer>: 250101
<question>: 250102
</s>: 1

Example 1:
Input IDs: [259, 2218, 5643, 66118, 31858, 13790, 402, 25191, 842, 267, 259, 12563, 548, 89539, 445, 105046, 122506, 556, 402, 41887, 343, 341, 12563, 1021, 9674, 259, 161017, 21128, 1093, 632, 343, 259, 152997, 4326, 2042, 259, 20380, 1189, 25543, 40730, 140296, 1093, 632, 341, 96178, 16208, 7123, 32339, 184000, 259, 131419, 7139, 5587, 632, 343, 105662, 5722, 63577, 18527, 16208, 7123, 259, 28240, 842, 343, 341, 12563, 52949, 7123, 259, 9793, 913, 5643, 149673, 16858, 24320, 974, 12306, 259, 271, 1923, 152029, 312, 3691, 7360, 60400, 343, 341, 12563, 548, 33478, 259, 9793, 94265, 52864, 1491, 92209, 445, 343, 341, 89539, 636, 259, 53442, 12592, 16208, 7123, 402, 20380, 78557, 70986, 9554, 41216, 343, 341, 12563, 94265, 52864, 24985, 159290, 10866, 766, 30891, 556, 7360, 60400, 343, 341, 12563, 26635, 17902, 402, 6837, 766, 16637, 5373, 2042, 259, 137310, 25625, 3572, 11013, 226086, 766, 87544, 3

In [19]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100
)

In [20]:
import torch
torch.cuda.empty_cache()


In [21]:
# Clean up previous outputs if any
!rm -rf /kaggle/working/*


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
training_args = Seq2SeqTrainingArguments(
    # output_dir="./mt5-qg",
    # eval_strategy="epoch",
    # # learning_rate=3e-5,
    # per_device_train_batch_size=2,  # Reduced for memory
    # per_device_eval_batch_size=2,
    # # weight_decay=0.01,
    # learning_rate=3e-4,
    # weight_decay=0.0,
    # save_total_limit=1,
    # num_train_epochs=5,
    # predict_with_generate=True,
    # fp16=False,
    # logging_steps=50,
    # save_strategy="epoch",
    # generation_num_beams=4,
    # report_to="wandb",
    # load_best_model_at_end=True
    output_dir="./arabart-qg",  # Not used for saving during training anymore
    eval_strategy="no",
    save_strategy="no",                       # ✅ Don't save during training
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    num_train_epochs=12,
    learning_rate=3e-5,
    weight_decay=0.01,
    generation_max_length=128,
    generation_num_beams=4,
    report_to="none",
    fp16=True
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qg_datasets["train"],
    eval_dataset=tokenized_qg_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_19/1214606555.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [23]:
# Start training
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,12.613400
1000,1.568800
1500,0.461600
2000,0.352700
2500,0.395300
3000,0.343100
3500,0.307200
4000,0.290700
4500,0.283200
5000,0.270600


TrainOutput(global_step=14112, training_loss=0.7495305527364856, metrics={'train_runtime': 16010.2382, 'train_samples_per_second': 3.524, 'train_steps_per_second': 0.881, 'total_flos': 6.765497893571789e+16, 'train_loss': 0.7495305527364856, 'epoch': 12.0})

In [24]:
model.save_pretrained("./arabart_finetuned/final_model")
tokenizer.save_pretrained("./arabart_finetuned/final_model")

('./arabart_finetuned/final_model/tokenizer_config.json',
 './arabart_finetuned/final_model/special_tokens_map.json',
 './arabart_finetuned/final_model/spiece.model',
 './arabart_finetuned/final_model/added_tokens.json',
 './arabart_finetuned/final_model/tokenizer.json')

In [25]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./finetuned-AraT5-QG",
#     evaluation_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     predict_with_generate=True,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=5,
#     fp16=False,  # Disable mixed precision for debugging
#     generation_num_beams=3,
#     gradient_accumulation_steps=4,
#     # gradient_checkpointing=True,
#     save_strategy="epoch",
#     logging_dir="./logs",
# )
# # print(training_args)  # Verify the training arguments


# # Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_qg_datasets["train"],
#     eval_dataset=tokenized_qg_datasets["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# # Start training
# trainer.train()